# Credit Card Fraud Detection - Sparkov Dataset

In [14]:
import sys
sys.path.append('../')

import pandas as pd
import numpy as np

from offline_preprocessing import convert_bucket_feat, convert_categorical_feat, time_projection, zscore_clipping


In [15]:
df = pd.concat([pd.read_csv('../../data/raw/fraudTrain.csv'), pd.read_csv('../../data/raw/fraudTest.csv')], ignore_index=True)
#df = pd.read_csv('../../datasets/raw/fraudTest.csv')
df.head()

,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,0,2019-01-01 00:00:18,2703186189652095,"fraud_Rippin, Kub and Mann",misc_net,4.97,Jennifer,Banks,F,561 Perry Cove,...,36.0788,-81.1781,3495,"Psychologist, counselling",1988-03-09,0b242abb623afc578575680df30655b9,1325376018,36.011293,-82.048315,0
1,1,2019-01-01 00:00:44,630423337322,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,Stephanie,Gill,F,43039 Riley Greens Suite 393,...,48.8878,-118.2105,149,Special educational needs teacher,1978-06-21,1f76529f8574734946361c461b024d99,1325376044,49.159047,-118.186462,0
2,2,2019-01-01 00:00:51,38859492057661,fraud_Lind-Buckridge,entertainment,220.11,Edward,Sanchez,M,594 White Dale Suite 530,...,42.1808,-112.2620,4154,Nature conservation officer,1962-01-19,a1a22d70485983eac12b5b88dad1cf95,1325376051,43.150704,-112.154481,0
3,3,2019-01-01 00:01:16,3534093764340240,"fraud_Kutch, Hermiston and Farrell",gas_transport,45.00,Jeremy,White,M,9443 Cynthia Court Apt. 038,...,46.2306,-112.1138,1939,Patent attorney,1967-01-12,6b849c168bdad6f867558c3793159a81,1325376076,47.034331,-112.561071,0
4,4,2019-01-01 00:03:06,375534208663984,fraud_Keeling-Crist,misc_pos,41.96,Tyler,Garcia,M,408 Bradley Rest,...,38.4207,-79.4629,99,Dance movement psychotherapist,1986-03-28,a41d7549acf90789359a9aa5346dcb46,1325376186,38.674999,-78.632459,0


## Pre-Processing

What we don't need to do:
- Deal with missing values, there are none in the dataset
- Deal with duplicates, there are none in the dataset

#### Time feature processing

Generate client age:

In [16]:
timestamp = pd.to_datetime(df['trans_date_trans_time'])
df['age'] = ((timestamp - pd.to_datetime(df['dob']))/np.timedelta64(1, 'Y'))
df['age'] = df['age'].astype(int)
df['age']

0          30
1          40
2          56
3          52
4          32
           ..
1852389    54
1852390    21
1852391    39
1852392    55
1852393    27
Name: age, Length: 1852394, dtype: int64

Generate time between last transaction

In [17]:
def calculate_timedelta(x):
    if x['cc_num'] != x['previous_card']:
        return -1
    else:
        return x['unix_time'] - x['previous_timestamp']

df = df.sort_values(['cc_num', 'unix_time'], ascending=[True, True])

df['trans_timedelta'] = np.nan
df['previous_timestamp'] = df['unix_time'].shift()
df['previous_card'] = df['cc_num'].shift()

df['trans_timedelta'] = df.apply(calculate_timedelta, axis=1)
        
df.drop(['previous_card', 'previous_timestamp'],axis=1,inplace=True)


Convert transaction date

In [18]:
timestamp.dt.month

0           1
1           1
2           1
3           1
4           1
           ..
1852389    12
1852390    12
1852391    12
1852392    12
1852393    12
Name: trans_date_trans_time, Length: 1852394, dtype: int32

In [19]:
df['trans_month_sin'], df['trans_month_cos'] = time_projection(timestamp.dt.month, 30)
df['trans_week_sin'], df['trans_week_cos'] = time_projection(timestamp.dt.day_of_week, 7)
df['trans_hour_sin'], df['trans_hour_cos'] = time_projection(timestamp.dt.hour, 24)

#### Sort dataset back to time

In [20]:
df = df.sort_values('unix_time', ascending=True)
df.reset_index(drop=True)
df

,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,...,merch_long,is_fraud,age,trans_timedelta,trans_month_sin,trans_month_cos,trans_week_sin,trans_week_cos,trans_hour_sin,trans_hour_cos
0,0,2019-01-01 00:00:18,2703186189652095,"fraud_Rippin, Kub and Mann",misc_net,4.97,Jennifer,Banks,F,561 Perry Cove,...,-82.048315,0,30,-1.0,0.207912,0.978148,0.781831,0.623490,0.000000,1.000000
1,1,2019-01-01 00:00:44,630423337322,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,Stephanie,Gill,F,43039 Riley Greens Suite 393,...,-118.186462,0,40,-1.0,0.207912,0.978148,0.781831,0.623490,0.000000,1.000000
2,2,2019-01-01 00:00:51,38859492057661,fraud_Lind-Buckridge,entertainment,220.11,Edward,Sanchez,M,594 White Dale Suite 530,...,-112.154481,0,56,-1.0,0.207912,0.978148,0.781831,0.623490,0.000000,1.000000
3,3,2019-01-01 00:01:16,3534093764340240,"fraud_Kutch, Hermiston and Farrell",gas_transport,45.00,Jeremy,White,M,9443 Cynthia Court Apt. 038,...,-112.561071,0,52,-1.0,0.207912,0.978148,0.781831,0.623490,0.000000,1.000000
4,4,2019-01-01 00:03:06,375534208663984,fraud_Keeling-Crist,misc_pos,41.96,Tyler,Garcia,M,408 Bradley Rest,...,-78.632459,0,32,-1.0,0.207912,0.978148,0.781831,0.623490,0.000000,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1852389,555714,2020-12-31 23:59:07,30560609640617,fraud_Reilly and Sons,health_fitness,43.77,Michael,Olson,M,558 Michael Estates,...,-91.333331,0,54,16630.0,0.587785,-0.809017,0.433884,-0.900969,-0.258819,0.965926
1852390,555715,2020-12-31 23:59:09,3556613125071656,fraud_Hoppe-Parisian,kids_pets,111.84,Jose,Vasquez,M,572 Davis Mountains,...,-96.186633,0,21,9743.0,0.587785,-0.809017,0.433884,-0.900969,-0.258819,0.965926
1852391,555716,2020-12-31 23:59:15,6011724471098086,fraud_Rau-Robel,kids_pets,86.88,Ann,Lawson,F,144 Evans Islands Apt. 683,...,-119.715054,0,39,724.0,0.587785,-0.809017,0.433884,-0.900969,-0.258819,0.965926
1852392,555717,2020-12-31 23:59:24,4079773899158,fraud_Breitenberg LLC,travel,7.99,Eric,Preston,M,7020 Doyle Stream Apt. 951,...,-117.080888,0,55,12025.0,0.587785,-0.809017,0.433884,-0.900969,-0.258819,0.965926


#### Binazrizing gender column:

In [21]:
def gender_binarizer(x):
    if x=='F':
        return 1
    if x=='M':
        return -1
    
df['gender'] = df['gender'].transform(gender_binarizer)
df['gender']

0          1
1          1
2         -1
3         -1
4         -1
          ..
1852389   -1
1852390   -1
1852391    1
1852392   -1
1852393   -1
Name: gender, Length: 1852394, dtype: int64

#### Feature Bucketing

In [22]:
df['lat'] = convert_bucket_feat(df['lat'])
df['long'] = convert_bucket_feat(df['long'])
df['merch_lat'] = convert_bucket_feat(df['merch_lat'])
df['merch_long'] = convert_bucket_feat(df['merch_long'])

/home/nancio/Documents/repos/Thesis/src/data_processing/offline_preprocessing.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  column.loc[(column >= lower_limit) & (column <= upper_limit)] = (upper_quantile-0.01) * 100
/home/nancio/Documents/repos/Thesis/src/data_processing/offline_preprocessing.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  column.loc[(column >= lower_limit) & (column <= upper_limit)] = (upper_quantile-0.01) * 100
/home/nancio/Documents/repos/Thesis/src/data_processing/offline_preprocessing.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See

#### Categorical feature processing

In [23]:
categorical_feat = {
    'lat': 100,
    'long': 100,
    'merch_lat': 100,
    'merch_long': 100,
    'category': 14,
    'cc_num': 999,
    'merchant': 693
}

for key, value in categorical_feat.items():
    df[key] = convert_categorical_feat(df[key], value)

#### Feature Scaling

In [24]:
features = ('amt', 'trans_timedelta', 'trans_month_sin', 'trans_month_cos', 'trans_week_sin', 'trans_week_cos', 'trans_hour_sin', 'trans_hour_cos', 'city_pop')
#, 'trans_timedelta', 
for feat in features:
    df[feat] = zscore_clipping(df[feat])

#### Drop unecesary columns

In [25]:
df.drop(['Unnamed: 0', 'first', 'last', 'job', 'street', 'state', 'trans_num', 'zip', 'city', 'dob'],axis=1,inplace=True)
df.head()

,trans_date_trans_time,cc_num,merchant,category,amt,gender,lat,long,city_pop,unix_time,...,merch_long,is_fraud,age,trans_timedelta,trans_month_sin,trans_month_cos,trans_week_sin,trans_week_cos,trans_hour_sin,trans_hour_cos
0,2019-01-01 00:00:18,146,585,12,-0.408741,1,29.0,93.0,-0.282429,1325376018,...,25.0,0,30,-0.634966,-2.500818,1.529069,1.250178,0.671264,0.197699,1.433818
1,2019-01-01 00:00:44,51,105,2,0.233378,1,62.0,50.0,-0.293527,1325376044,...,79.0,0,40,-0.634966,-2.500818,1.529069,1.250178,0.671264,0.197699,1.433818
2,2019-01-01 00:00:51,860,366,7,0.942184,-1,69.0,95.0,-0.280243,1325376051,...,73.0,0,56,-0.634966,-2.500818,1.529069,1.250178,0.671264,0.197699,1.433818
3,2019-01-01 00:01:16,696,39,1,-0.157381,-1,37.0,66.0,-0.287590,1325376076,...,72.0,0,52,-0.634966,-2.500818,1.529069,1.250178,0.671264,0.197699,1.433818
4,2019-01-01 00:03:06,195,521,11,-0.176470,-1,62.0,79.0,-0.293693,1325376186,...,93.0,0,32,-0.634966,-2.500818,1.529069,1.250178,0.671264,0.197699,1.433818


#### Save full modified dataset

In [27]:
df.to_csv("../../data/modified/modified_sparkov_dataframe.csv")git